<a href="https://colab.research.google.com/github/MikolajMichaluk/Algorithms/blob/main/Programming_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The file contains the adjacency list representation of a simple undirected graph. There are 200 vertices labeled 1 to 200. The first column in the file represents the vertex label, and the particular row (other entries except the first column) tells all the vertices that the vertex is adjacent to. So for example, the 6^{th}6 
th
  row looks like : "6	155	56	52	120	......". This just means that the vertex with label 6 is adjacent to (i.e., shares an edge with) the vertices with labels 155,56,52,120,......,etc

Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut.  (HINT: Note that you'll have to figure out an implementation of edge contractions.  Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction.  But you should also think about more efficient implementations.)   (WARNING: As per the video lectures, please make sure to run the algorithm many times with different random seeds, and remember the smallest cut that you ever find.)  Write your numeric answer in the space provided.  So e.g., if your answer is 5, just type 5 in the space provided.

In [517]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kargerMinCut.txt to kargerMinCut (1).txt
User uploaded file "kargerMinCut.txt" with length 18570 bytes


In [518]:
from random import choices, randint
import math

class Graph:
  def __init__(self, graph_path):
    self._file_name = None

    if type(graph_path) != dict:
      self._file_name = graph_path
      self._graph = self.read_graph_from_txt()
    else:
      self._graph = self.graph_from_dict(graph_path)

  def choose_random_edge(self):

    graph_size = self.return_number_of_vertices()

    vertices = [vertex for vertex in self._graph]
    edges = [self._graph[vertex] for vertex in self._graph]
    vertices_weight = [len(edge) for edge in edges]

    vertex = choices(vertices, vertices_weight)
    edge = self._graph[vertex[0]][randint(0, len(self._graph[vertex[0]])-1)]
    
    return vertex[0] , edge


  def contract(self, vertex_1, vertex_2):
    if self.has_edge(vertex_1, vertex_2):
      self._graph[vertex_1].remove(vertex_2)
      self._graph[vertex_2].remove(vertex_1)
      self._graph[vertex_1].extend(self._graph[vertex_2])
      self._graph.pop(vertex_2)

      for vertex in self._graph:
        if vertex != vertex_1 or vertex != vertex_2:
          for i in range(0, len(self._graph[vertex])):
            if self._graph[vertex][i] == vertex_2:
              self._graph[vertex][i] = vertex_1

      while vertex_1 in self._graph[vertex_1]:
        self._graph[vertex_1].remove(vertex_1)

    else:
      print(f"{vertex_1} and {vertex_2} not connected")

  def has_edge(self, vertex_1, vertex_2):
    if vertex_2 in self._graph[vertex_1]:
      return True
    else:
      return False

  def read_graph_from_txt(self):
    graph = dict()
    with open(self._file_name) as txt_file:
      for line in txt_file:
        line = line.split()
        graph.update({line[0] : line[1:]})
    return graph
  
  def graph_from_dict(self, dict_graph):
    return dict_graph

  def __repr__(self):
    repr = ""
    for vertex in self._graph:
      repr += f"vertex {vertex} edges {self._graph[vertex]} \n"
    return repr

  def get_first_vertex(self):
    for vertex in self._graph:
      return vertex
  
  def get_first_vertex_edges_number(self):
    return len(self._graph[self.get_first_vertex()])

  def return_number_of_vertices(self):
    return len(self._graph)

In [519]:
from copy import deepcopy

class KargerMinCut:
  def __init__(self, graph,iteration = 10):
    self._iteration = iteration
    self._graph = graph
    self._minCutArray = []

  def _kargerMinCut(self):
    temp_graph = copy.deepcopy(self._graph)

    while temp_graph.return_number_of_vertices() > 2:
      rand_vertex, rand_edge = temp_graph.choose_random_edge()
      temp_graph.contract(rand_vertex, rand_edge)

    cutMin = temp_graph.get_first_vertex_edges_number()

    return cutMin

  def kargerMinCut(self):
    for i in range(0, self._iteration):
      self._kargerMinCut()
      self._minCutArray.append(self._kargerMinCut())

    return min(self._minCutArray)


In [520]:
graph = Graph({1: [2, 2, 3, 4], 2 : [1, 1,3], 3 : [1, 2, 4], 4: [1,3]})
graph

vertex 1 edges [2, 2, 3, 4] 
vertex 2 edges [1, 1, 3] 
vertex 3 edges [1, 2, 4] 
vertex 4 edges [1, 3] 

In [521]:
KargerMinCut(graph, 10).kargerMinCut()

2

In [522]:
graph = Graph("kargerMinCut.txt")

In [523]:
KargerMinCut(graph, 100).kargerMinCut()

17